In [1]:
import pandas as pd
import spacy
from spacy import displacy

In [2]:
from collections import Counter
from string import punctuation

In [3]:
nlp = spacy.load("en_core_web_lg")

In [5]:
data = pd.read_json("reviews_with_entities_and_clean_content.json", orient='records')
print(data.head())

   reviewid                 title            artist  score          author  \
0     22703             mezzanine    massive attack    9.3     nate patrin   
1     22721          prelapsarian          krallice    7.9        zoe camp   
2     22659  all of them naturals      uranium club    7.3  david glickman   
3     22661           first songs  kleenex, liliput    9.0      jenn pelly   
4     22725             new start              taso    8.1    kevin lozano   

   review_publication_date       genre song_release_year  \
0            1483833600000  electronic              1998   
1            1483747200000       metal              2016   
2            1483747200000        rock              2016   
3            1483660800000        rock              2016   
4            1483660800000  electronic              2016   

                      record_label  \
0                           virgin   
1                        hathenter   
2  static shock,fashionable idiots   
3      kill rock s

In [5]:
artists = list(data['artist'])
for ar in artists[0:5]:
    l = ar.split(", ")
    print(l)

['massive attack']
['krallice']
['uranium club']
['kleenex', 'liliput']
['taso']


In [6]:
all_entities = list(data['entities'])
content = list(data['empty_content'])
artists = list(data['artist'])
titles = list(data['title'])
only_entities = []
empty_content = []
same = []
for i in range(len(content)):
    a = all_entities[i]
    c = content[i]
    if not a:
        
        empty_content.append(c)
        continue
    artist = artists[i]
    splitArtists = artist.split(", ")
    for ar in splitArtists:
        c = c.lower().replace(ar, '')
    title = titles[i]
    splitTitle = title.split(", ")
    for t in splitTitle:
        c = c.lower().replace(t, '')
    if content[i].lower() != c:
        same.append(i)
    empty_content.append(c)

print(len(same))

6350


In [8]:
data['Clean_Content']= empty_content

In [9]:
data

reviewid                        title            artist  score  \
0         22703                    mezzanine    massive attack    9.3   
1         22721                 prelapsarian          krallice    7.9   
2         22659         all of them naturals      uranium club    7.3   
3         22661                  first songs  kleenex, liliput    9.0   
4         22725                    new start              taso    8.1   
...         ...                          ...               ...    ...   
18412      1535               let us replay!           coldcut    8.9   
18413      1341                         1999           cassius    4.8   
18414      5376                  out of tune          mojave 3    6.3   
18415      2413  singles breaking up, vol. 1     don caballero    7.2   
18416      3723    left for dead in malaysia    neil hamburger    6.5   

                author  review_publication_date                    genre  \
0          nate patrin            1483833600000               electronic   
1             zoe camp            1483747200000                    metal   
2       david glickman            1483747200000                     rock   
3           jenn pelly            1483660800000                     rock   
4         kevin lozano            1483660800000               electronic   
...                ...                      ...                      ...   
18412  james p. wisdom             917308800000          jazz,electronic   
18413  james p. wisdom             917308800000               electronic   
18414   jason josephes             916099200000                     rock   
18415  james p. wisdom             916099200000  rock,metal,experimental   
18416   jason josephes             915494400000                     None   

      song_release_year                     record_label  \
0                  1998                           virgin   
1                  2016                        hathenter   
2                  2016  static shock,fashionable idiots   
3                  2016      kill rock stars,mississippi   
4                  2016                          teklife   
...                 ...                              ...   
18412                 0                       ninja tune   
18413                 0                      astralwerks   
18414                 0                              4ad   
18415              1999                     touch and go   
18416                 0                        drag city   

                                          review_content  ...  \
0      “Trip-hop” eventually became a ’90s punchline,...  ...   
1      Eight years, five albums, and two EPs in, the ...  ...   
2      Minneapolis’ Uranium Club seem to revel in bei...  ...   
3      Kleenex began with a crash. It transpired one ...  ...   
4      It is impossible to consider a given release b...  ...   
...                                                  ...  ...   
18412  The marketing guys of yer average modern megac...  ...   
18413  Well, it's been two weeks now, and I guess it'...  ...   
18414  Out of Tune is a Steve Martin album.  Yes, I'l...  ...   
18415  Well, kids, I just went back and re-read my re...  ...   
18416  Neil Hamburger's third comedy release is a des...  ...   

                                                  places  \
0      [Bristol, Jamaica, Bristol, Memphis, UK, Earth...   
1                 [New York, Prelapsarian, Prelapsarian]   
2                                 [Minneapolis, Detroit]   
3             [Zurich, Britain, London, LiLiPUT, Zurich]   
4      [San Francisco, the Bay Area, Taso, Taso, Miam...   
...                                                  ...   
18412                                                [ ]   
18413                                           [France]   
18414                                  [Seattle, Mojave]   
18415                                          [Singles]   
18416        [America, Kuala Lumpur, Malaysia, Malaysia

In [10]:
data.to_json('reviews_with_entities_and_clean_content_ETA.json', orient='records')

In [16]:
music_words= ['album', 'genre', 'music', 'albums', 'song', 'songs', 'genres', 'year', 'minutes', 'minute', 'years']
def get_hotwords(text, x):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] # 1
    doc = nlp(text.lower()) # 2
    for token in doc:
        # 3
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
            
        if(token.text in music_words):
            continue
            
        if(token.pos_ in pos_tag):
            result.append(token.text)
            
    out = Counter(result).most_common(x)
                
    return out

In [17]:
content = list(data['Clean_Content'])
titles = list(data['title'])
artist = list(data['artist'])
key_words = []
print(len(content))
for i in range(len(content)):
    print (i, end="\r")
    doc = nlp(content[i])
    result = get_hotwords(content[i], 25)
    key_words.append(result)

18417


In [18]:
data['keywords_clean'] = key_words

In [19]:
data.to_json('reviews_clean_content_and_key_words.json', orient='records')

In [21]:
print(data.head())

   reviewid                 title            artist  score          author  \
0     22703             mezzanine    massive attack    9.3     nate patrin   
1     22721          prelapsarian          krallice    7.9        zoe camp   
2     22659  all of them naturals      uranium club    7.3  david glickman   
3     22661           first songs  kleenex, liliput    9.0      jenn pelly   
4     22725             new start              taso    8.1    kevin lozano   

   review_publication_date       genre song_release_year  \
0            1483833600000  electronic              1998   
1            1483747200000       metal              2016   
2            1483747200000        rock              2016   
3            1483660800000        rock              2016   
4            1483660800000  electronic              2016   

                      record_label  \
0                           virgin   
1                        hathenter   
2  static shock,fashionable idiots   
3      kill rock s